In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
import pickle
import numpy as np

## Coco evaluation

In [2]:
#initialize COCO ground truth api
annType = 'bbox'
annFile = '/media/semyon/Data/Documents/coco/annotations/instances_val2017_bbox.json'
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=0.64s)
creating index...
index created!


In [3]:
len(cocoGt.anns)

36781

In [4]:
#initialize COCO detections api
resFile = '/media/semyon/Data/Documents/coco/detections_coco.json'
cocoDt=cocoGt.loadRes(resFile)

Loading and preparing results...
DONE (t=0.35s)
creating index...
index created!


In [5]:
with open(resFile, 'r') as input_file:
    yolo_dt = json.load(input_file)
for i in range(1, len(cocoDt.anns) + 1):
    c_a = cocoDt.anns[i]
    y_a = yolo_dt[i - 1]
    c_b = np.array(c_a['bbox'])
    y_b = np.array(y_a['bbox'])
    if not (c_b == y_b).all():
        print(i)

In [6]:
imgIds=sorted(cocoGt.getImgIds())
print(len(imgIds))

5000


In [7]:
coco_cat_ids_path = '/media/semyon/Data/Documents/coco/coco_cat_ids.json'
with open(coco_cat_ids_path, 'r') as input_file:
    coco_cat_ids = json.load(input_file)
print(coco_cat_ids)
print(len(coco_cat_ids))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90]
80


In [8]:
cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds
cocoEval.params.catIds = coco_cat_ids

In [9]:
# running evaluation
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=14.06s).
Accumulating evaluation results...
DONE (t=2.25s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.292
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.505
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.075
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.246
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.451
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.247
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.340
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.344
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.103
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=10

## Extract data

In [14]:
evalImg_all_100 = [e for e in cocoEval.evalImgs if e is not None and (e['maxDet'] == 100 and e['aRng'][1] == 1e10 and e['aRng'][0] == 0)]

In [18]:
evalImg_m_100 = [e for e in cocoEval.evalImgs if e is not None and (e['maxDet'] == 100 and e['aRng'][1] == 96 ** 2 and e['aRng'][0] == 32 ** 2)]

In [15]:
evalImg = evalImg_all_100

In [17]:
matches = []
for e in evalImg:
    match = {
        'image_id': e['image_id'],
        'category_id': e['category_id'],
        'target_ids': e['gtIds'],
        'pred_ids': e['dtIds'],
        'matched': e['dtMatches'][0],
        'scores': e['dtScores']
    }
    matches.append(match)
coco_matches_path = '/media/semyon/Data/Documents/coco/coco_matches.pkl'
with open(coco_matches_path, 'wb') as output_file:
    pickle.dump(matches, output_file)

In [23]:
from collections import defaultdict
evalImg_dict = defaultdict(list)
for e in evalImg_m_100:
    evalImg_dict[e['category_id']].append(e)
for c in evalImg_dict:
    print(c)
    gti = np.concatenate([e['gtIgnore'] for e in evalImg_dict[c]])
    print((gti == 0).sum(), len(gti))

1
3723 10777
2
123 314
3
547 1918
4
138 367
5
43 143
6
82 283
7
18 190
8
164 414
9
135 424
10
99 634
11
30 101
13
21 75
14
21 60
15
132 411
16
85 427
17
37 202
18
62 218
19
98 272
20
159 354
21
123 372
22
96 252
23
13 71
24
95 266
25
97 232
27
135 371
28
169 407
31
204 540
32
77 252
33
118 299
34
40 115
35
59 241
36
24 69
37
27 260
38
81 327
39
61 145
40
52 148
41
84 179
42
118 267
43
125 225
44
328 1013
46
116 341
47
351 895
48
91 215
49
96 325
50
70 253
51
234 623
52
173 370
53
91 236
54
48 177
55
116 285
56
189 312
57
157 365
58
33 125
59
80 284
60
96 328
61
122 310
62
829 1771
63
58 261
64
189 342
65
14 163
67
173 695
70
47 179
72
106 288
73
80 231
74
44 106
75
60 283
76
78 153
77
78 262
78
33 55
79
51 143
80
5 9
81
98 225
82
28 126
84
239 1129
85
99 267
86
98 274
87
9 36
88
49 190
89
3 11
90
18 57
